In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

%matplotlib notebook

Loading the dataset

In [2]:
df = pd.read_csv("paysim1/dataset.csv")

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


Renaming columns to make them consistent

In [4]:
df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', 
                   'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'}, inplace=True)

In [5]:
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


Checking what type of transactions are fraudulent?

In [6]:
type_fraud = df[['type', 'isFraud']]
type_fraud = type_fraud[type_fraud['isFraud'] ==1]
type_fraud.groupby('type').count()

,isFraud
type,
CASH_OUT,4116
TRANSFER,4097


Dropping all rows that are not of type 'CASH_OUT' or 'TRANSFER' as none of the fraud transactions are of the other three types. So, in a real world datset, we can preprocess and declare all other tree types as non-fraud.

In [7]:
df = df[(df['type'] == 'CASH_OUT') | (df['type'] == 'TRANSFER')]

In [8]:
len(df)

2770409

Checking if any of the transactions originate from a merchant?

In [9]:
print("Any merchants as originator of transactions? {}".format(df.nameOrig.str.contains('M').any()))

Any merchants as originator of transactions? False


In [10]:
print("Any merchants as recepient of transactions? {}".format(df.nameDest.str.contains('M').any()))

Any merchants as recepient of transactions? False


The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. We can check what is the ratios of missing values for fraud and genuine transactions?

In [22]:
Y = df['isFraud']
fraud = df.loc[Y == 1]
genuine = df.loc[Y == 0]
print('Fraction of fraud transactions with \'oldBalanceDest\' = \'newBalanceDest\' = 0' + 
      ' although the transacted \'amount\' is non-zero is: {}'.format(
          len(fraud.loc[(fraud.oldBalanceDest == 0) & (fraud.newBalanceDest == 0) 
                        & (fraud.amount)]) / (1.0 * len(fraud))))

print('\nFraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(genuine.loc[(genuine.oldBalanceDest == 0) & \
(genuine.newBalanceDest == 0) & (genuine.amount)]) / (1.0 * len(genuine))))

print('\nFraction of fraud transactions with \'oldBalanceOrig\' = \'newBalanceOrig\' = 0' + 
      ' although the transacted \'amount\' is non-zero is: {}'.format(
          len(fraud.loc[(fraud.oldBalanceOrig == 0) & (fraud.newBalanceOrig == 0) 
                        & (fraud.amount)]) / (1.0 * len(fraud))))

print('\nFraction of genuine transactions with \'oldBalanceOrig\' = \
newBalanceOrig\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(genuine.loc[(genuine.oldBalanceOrig == 0) & \
(genuine.newBalanceOrig == 0) & (genuine.amount)]) / (1.0 * len(genuine))))

Fraction of fraud transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4955558261293072

Fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0006176245277308345

Fraction of fraud transactions with 'oldBalanceOrig' = 'newBalanceOrig' = 0 although the transacted 'amount' is non-zero is: 0.0030439547059539756

Fraction of genuine transactions with 'oldBalanceOrig' = newBalanceOrig' = 0 although the transacted 'amount' is non-zero is: 0.4737321319703598


Since the destination account balances being zero is a strong indicator of fraud, we do not impute the account balance (before the transaction is made) with a statistic or from a distribution with a subsequent adjustment for the amount transacted. Doing so would mask this indicator of fraud and make fraudulent transactions appear genuine. Instead, below we replace the value of 0 with -1 which will be more useful to a suitable machine-learning (ML) algorithm detecting fraud.

In [27]:
df.loc[(df.oldBalanceDest == 0) & (df.newBalanceDest == 0) 
        & (df.amount != 0), ['oldBalanceDest', 'newBalanceDest']] = -1

The data also has several transactions with zero balances in the originating account both before and after a non-zero amount is transacted. In this case, the fraction of such transactions is much smaller in fraudulent (0.3%) compared to genuine transactions (47%). Once again, from similar reasoning as above, instead of imputing a numerical value we replace the value of 0 with a null value.

In [28]:
df.loc[(df.oldBalanceOrig == 0) & (df.newBalanceOrig == 0) 
        & (df.amount != 0), ['oldBalanceOrig', 'newBalanceOrig']] = np.nan

In [29]:
df['errorOrig'] = df['newBalanceOrig'] + df['amount'] - df['oldBalanceOrig']
df['errorDest'] = df['oldBalanceDest'] + df['amount'] - df['newBalanceDest']

In [31]:
print("Account not balanced in Orig:", len(df[df['errorOrig'] != 0])/ len(df))
print("Account not balanced in Dest:", len(df[df['errorDest'] != 0])/ len(df))

Account not balanced in Orig: 0.9097685576389624
Account not balanced in Dest: 0.4209429004887004


Checking if 'isFlaggedFraud' is a useful indicator of fraud?

In [34]:
print("Proportion of Fraudulent transactions which were flagged", sum(fraud.isFlaggedFraud == 1) /len(fraud))
print("Proportion of Genuine transactions which were flagged", sum(genuine.isFlaggedFraud == 1) /len(genuine))

Proportion of Fraudulent transactions which were flagged 0.0019481310118105442
Proportion of Genuine transactions which were flagged 0.0


In [35]:
np.corrcoef(df.isFlaggedFraud, df.isFraud)

array([[1.        , 0.04407229],
       [0.04407229, 1.        ]])

Dropping useless columns: nameOrig, nameDest, isFlaggedFraud

In [36]:
df = pd.concat([df, pd.get_dummies(df['type'], prefix='type')], axis=1)
df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud', 'type'], axis=1)

In [37]:
df.head()

,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,errorOrig,errorDest,type_CASH_OUT,type_TRANSFER
2,1,181.00,181.0,0.0,-1.0,-1.00,1,0.00,181.0,0,1
3,1,181.00,181.0,0.0,21182.0,0.00,1,0.00,21363.0,1,0
15,1,229133.94,15325.0,0.0,5083.0,51513.44,0,213808.94,182703.5,1,0
19,1,215310.30,705.0,0.0,22425.0,0.00,0,214605.30,237735.3,0,1
24,1,311685.89,10835.0,0.0,6267.0,2719172.89,0,300850.89,-2401220.0,0,1


In [38]:
df['hour'] = df.step % 24

In [39]:
df.head()

,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,errorOrig,errorDest,type_CASH_OUT,type_TRANSFER,hour
2,1,181.00,181.0,0.0,-1.0,-1.00,1,0.00,181.0,0,1,1
3,1,181.00,181.0,0.0,21182.0,0.00,1,0.00,21363.0,1,0,1
15,1,229133.94,15325.0,0.0,5083.0,51513.44,0,213808.94,182703.5,1,0,1
19,1,215310.30,705.0,0.0,22425.0,0.00,0,214605.30,237735.3,0,1,1
24,1,311685.89,10835.0,0.0,6267.0,2719172.89,0,300850.89,-2401220.0,0,1,1


In [43]:
fraud = df[Y==1]
fraud.groupby('hour').step.count()

hour
0     300
1     358
2     372
3     326
4     274
5     366
6     358
7     328
8     368
9     341
10    375
11    324
12    339
13    346
14    353
15    341
16    345
17    353
18    343
19    342
20    340
21    347
22    351
23    323
Name: step, dtype: int64